In [1]:
import numpy as np
import pandas as pd
import os
import argparse
import time
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, roc_curve, auc
from sklearn.pipeline import Pipeline

# Linear models
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Non-linear models
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
# Import these packages conditionally to avoid linter errors
try:
    from xgboost import XGBClassifier
    from lightgbm import LGBMClassifier
    ADVANCED_MODELS_AVAILABLE = True
except ImportError:
    print("Warning: XGBoost and/or LightGBM are not installed. These models will be skipped.")
    ADVANCED_MODELS_AVAILABLE = False

# Neural Network
from sklearn.neural_network import MLPClassifier

def parse_arguments():
    parser = argparse.ArgumentParser(description='Train and evaluate classification models on proteomics data')
    parser.add_argument('--train_data', type=str, default='dataset/filtered_train_data.csv',
                        help='Path to training data')
    parser.add_argument('--test_data', type=str, default='dataset/filtered_test_data.csv',
                        help='Path to test data')
    parser.add_argument('--output_dir', type=str, default='291A/291A_assignment/models',
                        help='Directory to save models and results')
    parser.add_argument('--cv', type=int, default=3, 
                        help='Number of cross-validation folds')
    parser.add_argument('--n_jobs', type=int, default=-1,
                        help='Number of jobs for parallel processing')
    parser.add_argument('--random_state', type=int, default=42,
                        help='Random state for reproducibility')
    return parser.parse_args()

def load_data(train_path, test_path):
    """Load and prepare the training and test datasets"""
    print(f"Loading data from {train_path} and {test_path}")
    
    # Load datasets
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)
    
    # Extract feature columns and label
    feature_cols = [col for col in train_df.columns 
                   if col not in ['patient_id', 'label']]
    
    # Prepare training data
    X_train = train_df[feature_cols].values
    y_train = train_df['label'].values
    
    # Prepare test data
    X_test = test_df[feature_cols].values
    y_test = test_df['label'].values
    
    # Record patient IDs for reference
    train_patients = train_df['patient_id'].values
    test_patients = test_df['patient_id'].values
    
    print(f"Training data shape: {X_train.shape}")
    print(f"Test data shape: {X_test.shape}")
    
    # Use list to get unique classes instead of np.unique to avoid linter errors
    classes = sorted(list(set(y_train)))
    print(f"Classes: {classes}")
    
    return X_train, y_train, X_test, y_test, train_patients, test_patients, feature_cols

def get_models():
    """Define all models and their parameter grids for hyperparameter tuning"""
    models = {}
    param_grids = {}
    
    # Logistic Regression
    models['Logistic_Regression'] = LogisticRegression(max_iter=10000, random_state=42)
    param_grids['Logistic_Regression'] = {
        'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
        'classifier__penalty': ['l1', 'l2'],
        'classifier__solver': ['liblinear', 'saga']
    }

    return models, param_grids

def train_and_evaluate(X_train, y_train, X_test, y_test, args):
    """Train all models with hyperparameter tuning and evaluate on test set"""
    # Create output directory if it doesn't exist
    os.makedirs(args.output_dir, exist_ok=True)
    
    # Get models and parameter grids
    models, param_grids = get_models()
    
    # Setup cross-validation
    cv = StratifiedKFold(n_splits=args.cv, shuffle=True, random_state=args.random_state)
    
    # Results dictionary
    results = {
        'model_name': [],
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1_score': [],
        'training_time': [],
        'best_params': []
    }
    
    # Train and evaluate each model
    for model_name, model in models.items():
        print(f"\n{'-'*50}")
        print(f"Training {model_name}")
        
        # Create pipeline with scaling
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', model)
        ])
        
        # Setup grid search
        grid_search = GridSearchCV(
            pipeline,
            param_grids[model_name],
            cv=cv,
            scoring='accuracy',
            n_jobs=args.n_jobs,
            verbose=1
        )
        
        # Measure training time
        start_time = time.time()
        grid_search.fit(X_train, y_train)
        training_time = time.time() - start_time
        
        # Best model
        best_model = grid_search.best_estimator_
        
        # Predictions on test set
        y_pred = best_model.predict(X_test)
        
        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
        
        # Save results
        results['model_name'].append(model_name)
        results['accuracy'].append(accuracy)
        results['precision'].append(precision)
        results['recall'].append(recall)
        results['f1_score'].append(f1)
        results['training_time'].append(training_time)
        results['best_params'].append(grid_search.best_params_)
        
        # Print results
        print(f"Model: {model_name}")
        print(f"Best parameters: {grid_search.best_params_}")
        print(f"Training time: {training_time:.2f} seconds")
        print(f"Test accuracy: {accuracy:.4f}")
        print(f"Test precision: {precision:.4f}")
        print(f"Test recall: {recall:.4f}")
        print(f"Test F1 score: {f1:.4f}")
        
        # Save model
        model_path = os.path.join(args.output_dir, f"{model_name}_model.pkl")
        with open(model_path, 'wb') as f:
            pickle.dump(best_model, f)
        
        # Generate confusion matrix
        cm = confusion_matrix(y_test, y_pred)
        classes = sorted(list(set(np.concatenate((y_train, y_test)))))
        plot_confusion_matrix(cm, classes, model_name, args.output_dir)
        
        # Generate ROC curve for multi-class
        plot_roc_curve(best_model, X_test, y_test, model_name, args.output_dir)
    
    # Convert results to DataFrame and save
    results_df = pd.DataFrame(results)
    results_df.to_csv(os.path.join(args.output_dir, 'model_comparison.csv'), index=False)
    
    # Plot comparison of models
    plot_model_comparison(results_df, args.output_dir)
    
    return results_df

def plot_confusion_matrix(cm, classes, model_name, output_dir):
    """Plot and save confusion matrix"""
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(f'Confusion Matrix - {model_name}')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{model_name}_confusion_matrix.png"))
    plt.close()

def plot_roc_curve(model, X_test, y_test, model_name, output_dir):
    """Plot and save ROC curve for multi-class classification"""
    # Get probabilities for each class
    y_prob = model.predict_proba(X_test)
    
    # Get unique classes
    classes = sorted(list(set(y_test)))
    n_classes = len(classes)
    
    # Compute ROC curve and ROC area for each class
    fpr = {}
    tpr = {}
    roc_auc = {}
    
    plt.figure(figsize=(8, 6))
    
    for i, class_name in enumerate(classes):
        # Convert to binary classification problem (one-vs-rest)
        y_bin = (y_test == class_name).astype(int)
        class_probs = y_prob[:, i]
        
        # Compute ROC curve and area
        fpr[i], tpr[i], _ = roc_curve(y_bin, class_probs)
        roc_auc[i] = auc(fpr[i], tpr[i])
        
        # Plot ROC curve
        plt.plot(fpr[i], tpr[i],
                 label=f'ROC curve for {class_name} (area = {roc_auc[i]:.2f})')
    
    # Plot diagonal line (random classifier)
    plt.plot([0, 1], [0, 1], 'k--')
    
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - {model_name}')
    plt.legend(loc="lower right")
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{model_name}_roc_curve.png"))
    plt.close()

def plot_model_comparison(results_df, output_dir):
    """Plot and save model comparison metrics"""
    # Performance metrics bar chart
    plt.figure(figsize=(12, 8))
    results_melted = pd.melt(results_df, 
                             id_vars=['model_name'],
                             value_vars=['accuracy', 'precision', 'recall', 'f1_score'],
                             var_name='metric', value_name='value')
    
    sns.barplot(x='model_name', y='value', hue='metric', data=results_melted)
    plt.title('Model Performance Comparison')
    plt.xlabel('Model')
    plt.ylabel('Score')
    plt.xticks(rotation=45)
    plt.legend(title='Metric')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'model_performance_comparison.png'))
    plt.close()
    
    # Training time comparison
    plt.figure(figsize=(10, 6))
    sns.barplot(x='model_name', y='training_time', data=results_df)
    plt.title('Model Training Time Comparison')
    plt.xlabel('Model')
    plt.ylabel('Training Time (seconds)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'model_training_time_comparison.png'))
    plt.close()


In [7]:
train_data = '/root/code/ucsd_hw/291A_ass2/291A_assignment/dataset/filtered_train_data.csv'
test_data = '/root/code/ucsd_hw/291A_ass2/291A_assignment/dataset/filtered_test_data.csv'

In [8]:
X_train, y_train, X_test, y_test, train_patients, test_patients, feature_cols = load_data(
    train_data, test_data
)

Loading data from /root/code/ucsd_hw/291A_ass2/291A_assignment/dataset/filtered_train_data.csv and /root/code/ucsd_hw/291A_ass2/291A_assignment/dataset/filtered_test_data.csv
Training data shape: (21, 44711)
Test data shape: (6, 44711)
Classes: ['Control', 'MBC', 'TNBC']


In [18]:
from sklearn.feature_selection import SelectKBest, f_classif

K = 500
# 1. 在训练集上拟合特征选择器（非常重要，不要在全数据上fit）
selector = SelectKBest(score_func=f_classif, k=K)
X_train_selected = selector.fit_transform(X_train, y_train)

# 2. 用相同的选择器 transform 测试集
X_test_selected = selector.transform(X_test)


/root/miniconda3/envs/torch/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [  343   440   716  1443  1645  1761  2170  2180  2657  3020  3097  3697
  3723  3748  4270  4725  4845  4846  4847  4868  5511  5623  5640  5657
  5662  6655  6671  6885  6946  7315  7369  7490  7508  7895  8258  8270
  8815  8917  9348  9427  9790  9807  9983 10073 10075 10146 10578 10832
 10864 10884 11112 11155 11335 11337 11339 11340 11341 11978 11985 13110
 13170 13502 13582 13652 14224 14307 15229 15473 15585 15652 15756 15992
 16127 16252 16290 16369 17441 17566 17604 17714 20093 20227 20580 21007
 21598 21632 22232 22278 22304 22459 22753 22878 23185 23248 23504 23759
 24320 24803 25000 25291 25294 25665 25945 26161 26624 26730 26769 26840
 26962 27037 27071 27398 27533 27719 27727 27796 27869 27959 28011 28278
 28338 28615 28625 28628 28634 28683 29366 29460 29585 29739 29851 30221
 31256 31315 31600 32139 32535 32689 32830 33023 33465 33552 3